## requirements.txt

In [1]:
# !pip freeze > requirements.txt
!pip install -r requirements.txt

## Загрузка библиотек и зависимостей

In [2]:
import time
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
import catboost
from catboost import Pool, CatBoostClassifier

from sklearn.metrics import roc_auc_score, roc_curve, f1_score, classification_report, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder

import tsfresh as tsf
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series

In [4]:
# Imbalanced Dependencies
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import RandomOverSampler, ADASYN, BorderlineSMOTE
from imblearn.ensemble import BalancedBaggingClassifier, EasyEnsembleClassifier
from imblearn.ensemble import RUSBoostClassifier, BalancedRandomForestClassifier

# Under/Over Sampling
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import RandomOverSampler, ADASYN, BorderlineSMOTE

## Загрузка данных

In [5]:
df_train = pd.read_parquet("DCS/Train/hackaton2023_train.gzip")
df_test = pd.read_parquet("DCS/Test/hackaton2023_test.gzip")
submit = pd.read_csv("DCS/submission.csv", sep=';')

df_train.shape, df_test.shape

((12129384, 9), (2498034, 7))

|     Название                |     Описание                                                                             |     Тип   переменной    |     Комментарий                |   |
|-----------------------------|------------------------------------------------------------------------------------------|-------------------------|--------------------------------|---|
|     customer_id             |     Идентификатор   клиента                                                              |     int                 |                                |   |
|     group_name              |     Группа:   train (обучение) – test(контроль)                                          |                         |                                |   |
|     revenue                 |     Выручка   от продажи блюда в заказе                                                  |     float               |                                |   |
|     startdatetime           |     Дата   и время продажи                                                               |     datetime            |                                |   |
|     dish_name               |     Название   блюда                                                                     |     string              |                                |   |
|     ownareaall_sqm          |     Площадь   ресторана                                                                  |     float               |                                |   |
|     format_name             |     Формат   ресторана                                                                   |                         |                                |   |
|     buy_post                |     Таргет   1: флаг оттока     (0 – отток, 1 – не отток)                                |     bool                |     Только   в train данных    |   |
|     date_diff_post          |     Таргет   2: количество дней между последней покупкой в прошлом и первой в будущем    |     int                 |     Только   в train данных    |   |

In [6]:
df_train.head(7)

,customer_id,date_diff_post,buy_post,group_name,revenue,startdatetime,dish_name,ownareaall_sqm,format_name
0,29891,9.0,1,train,69.99,2022-12-05 12:03:58,Кинг Фри станд,300.0,Отдельно стоящий без внешней зоны
1,29891,9.0,1,train,190.00,2022-12-05 12:03:58,Чикен Тар-Тар,300.0,Отдельно стоящий без внешней зоны
2,29891,9.0,1,train,9.99,2022-12-05 12:03:58,Соус Сырный,300.0,Отдельно стоящий без внешней зоны
3,29891,9.0,1,train,119.99,2022-12-05 12:03:58,Энергет.нап. Адреналин Раш,300.0,Отдельно стоящий без внешней зоны
4,29891,9.0,1,train,119.99,2022-12-05 14:28:35,Латте (СТАНД.),300.0,Отдельно стоящий без внешней зоны
5,29891,9.0,1,train,60.00,2022-12-15 00:37:19,Чизбургер,463.0,Отдельно стоящий с внешней зоной
6,29891,9.0,1,train,209.99,2022-12-15 00:37:19,Воппер Ролл,463.0,Отдельно стоящий с внешней зоной


## EDA

In [7]:
# Распределение таргета
target_d = pd.DataFrame(df_train['buy_post'].value_counts())
target_d

,count
buy_post,
1,9660867
0,2468517


In [8]:
# Проверка на новых пользователей в тестовой выборке
train_customers = set(df_train['customer_id'].unique().tolist())
test_customers = set(df_test['customer_id'].unique().tolist())

assert len(train_customers - test_customers) == len(train_customers), "Клиенты в трейне не повторяются!"
assert len(test_customers - train_customers) == len(test_customers), "Клиенты в тесте не повторяются!"

# Делаем вывод, что для валидации нужно брать клиентов не из обучающей выборке

## Минимальная обработка данных

In [9]:
# Объединяем данные
data = pd.concat([df_train, df_test])

In [10]:
# Сортируем по ID клиента и по дате транзакции
data = data.sort_values(by=['customer_id', 'startdatetime'], ascending=True)

In [11]:
train_data = data[data['group_name'] == 'train']
test_data = data[data['group_name'] == 'train']

In [12]:
# Данные о количестве строк по пользователям
train_customer_counts = train_data.groupby('customer_id', as_index=False).agg({'group_name': 'count'})
train_customer_counts['count_n'] = train_customer_counts['group_name']
train_customer_counts = train_customer_counts.drop(columns=['group_name'])

# Данные о количестве строк по пользователям
test_customer_counts = test_data.groupby('customer_id', as_index=False).agg({'group_name': 'count'})
test_customer_counts['count_n'] = test_customer_counts['group_name']
test_customer_counts = test_customer_counts.drop(columns=['group_name'])

```
# --- Сохранение данных в локальную базу данных
# Label-кодирование строковых данных и сохранение справочников id-наименование

encoder = LabelEncoder()
data['encode_format_name'] = encoder.fit_transform(data['format_name'])
data['encode_dish_name'] = encoder.fit_transform(data['dish_name'])

data['is_test'] = data['group_name'].map({'train': 0, 'test': 1})
data = data.drop(columns=['group_name'])

format_name = data[['encode_format_name', 'format_name']].drop_duplicates().to_parquet("format_name.parquet", index=False)
dish_name = data[['encode_dish_name', 'dish_name']].drop_duplicates().to_parquet("dish_name.parquet", index=False)
data = data.drop(columns=['format_name', 'dish_name'])

data.to_parquet("data.parquet", index=False)

In [13]:
# --- Собираем строки в заказы

# Группируем по колонкам и генерируем минимальные фичи
data = data.groupby(["customer_id", "startdatetime"], as_index=False).agg(
    churn=("buy_post", "last"),
    date_diff_post=("date_diff_post", "last"),
    buy_post=("buy_post", "last"),
    group_name=("group_name", "last"),
    # revenue_min=("revenue", "min"),
    revenue=("revenue", "sum"),
    # revenue_max=("revenue", "max"),
    ownareaall_sqm=("ownareaall_sqm", "last"),
    format_name=("format_name", "last")
)

In [14]:
# Приводим таргеты к целочисленным форматам
data['buy_post'] = data['buy_post'].astype('Int64')
data['date_diff_post'] = data['date_diff_post'].astype('Int64')

In [15]:
# Разделяем данные на обучающие и тестовые
df_train = data[data['group_name'] == "train"]
df_test = data[data['group_name'] == "test"]

In [16]:
# --- Baseline Features

# Считаем признаки через агрегации 
df_train = df_train.groupby("customer_id", as_index=False).agg(
    churn=("buy_post", "last"),
    revenue_mean=("revenue", "mean"),
    revenue_sum=("revenue", "sum"),
    revenue_max=("revenue", "max"),
    revenue_min=("revenue", "min"),
    revenue_std=("revenue", "std"),
    revenue_last=("revenue", "last"),
    ownareaall_sqm_mean=("ownareaall_sqm", "mean"),
    ownareaall_sqm_max=("ownareaall_sqm", "max"),
    ownareaall_sqm_min=("ownareaall_sqm", "min"),
    ownareaall_sqm_std=("ownareaall_sqm", "std"),
    ownareaall_sqm_last=("ownareaall_sqm", "last"),
    format_name_last=("format_name", "last"),
    startdatetime_count=("startdatetime", "count"),
    startdatetime_std=("startdatetime", "std")
)
df_train['startdatetime_std'] = df_train['startdatetime_std'].dt.days
df_train = df_train.merge(train_customer_counts, how='left', on='customer_id')

In [17]:
df_train.head(7)

,customer_id,churn,revenue_mean,revenue_sum,revenue_max,revenue_min,revenue_std,revenue_last,ownareaall_sqm_mean,ownareaall_sqm_max,ownareaall_sqm_min,ownareaall_sqm_std,ownareaall_sqm_last,format_name_last,startdatetime_count,startdatetime_std,count_n
0,29891,1,203.494000,5087.35,439.98,1.00,123.170275,439.98,449.960000,463.0,300.00,45.132656,463.0,Отдельно стоящий с внешней зоной,25,15,34
1,30477,1,227.024000,5675.60,499.95,44.99,124.933425,44.99,320.000000,320.0,320.00,0.000000,320.0,Отдельно стоящий без внешней зоны,25,18,61
2,31426,1,391.399583,9393.59,1079.97,1.00,334.849322,44.99,153.000000,153.0,153.00,0.000000,153.0,Фудкорт без туалета,24,14,86
3,44491,1,128.725000,514.90,344.97,49.97,144.471912,69.99,126.355000,139.0,88.42,25.290000,139.0,Отдельно стоящий без внешней зоны без туалета,4,13,10
4,44939,1,554.943333,1664.83,604.93,504.96,49.985000,504.96,179.513333,280.0,129.27,87.024006,280.0,Отдельно стоящий с внешней зоной,3,7,25
5,45006,0,522.613333,1567.84,961.89,195.98,395.184669,409.97,159.030000,246.2,105.20,76.183441,105.2,Фудкорт без туалета,3,10,19
6,45038,1,719.950000,2879.80,989.94,379.96,307.343022,539.96,169.400000,261.0,125.10,62.922757,131.5,Отдельно стоящий без внешней зоны,4,19,18


## Custom Train-Test Split

In [18]:
def split_by_client(df, test_size=0.5):
    """
        Метод разделения трейн теста, таким образом чтобы одинаковые клиенты не попадали в разные наборы
        и при этом сохранилась стратификация по данным
    """
    clients_target_1 = df[df["churn"] == 1]["customer_id"].unique()
    clients_t1_train, clients_t1_test = train_test_split(clients_target_1, test_size=test_size, shuffle=True, random_state=53)
    
    clients_target_0 = df[df["churn"] == 0]["customer_id"].unique()
    clients_t0_train, clients_t0_test = train_test_split(clients_target_0, test_size=test_size, shuffle=True, random_state=53)
    
    clients_t0_train = list(set(clients_t0_train) - set(clients_t1_test))
    clients_t0_test = list(set(clients_t0_test) - set(clients_t1_train))
    
    train = pd.concat([df[(df['customer_id'].isin(clients_t0_train))], df[(df['customer_id'].isin(clients_t1_train))] ] )
    test = pd.concat([df[(df['customer_id'].isin(clients_t0_test))], df[(df['customer_id'].isin(clients_t1_test))]])
    
    # train = train.drop_duplicates(subset=["report_date", "customer_id"])
    # test = test.drop_duplicates(subset=["report_date", "customer_id"])
    
    return train, test

In [19]:
train_data, val_data = split_by_client(df_train, test_size=0.1)
# val_data, test_data = split_by_client(val_data, test_size=0.5)

# Проверяем, что нет лика данных по клиентам между трайн/вал/тест
assert len(set(train_data["customer_id"]) & set(val_data["customer_id"])) == 0, "Лик train val"
# assert len(set(train_data["customer_id"]) & set(test_data["customer_id"])) == 0, "Лик train test"
# assert len(set(test_data["customer_id"]) & set(val_data["customer_id"])) == 0, "Лик test val"

train_data.shape, test_data.shape  # , val_data.shape

((449999, 17), (12129384, 9))

In [20]:
# Проверяем дисбаланс разбиения train/val/test
train_data["churn"].value_counts(), val_data["churn"].value_counts()  # , test_data["churn"].value_counts()

(churn
 1    323185
 0    126814
 Name: count, dtype: Int64,
 churn
 1    35910
 0    14091
 Name: count, dtype: Int64)

In [21]:
train_data.dtypes

customer_id              int64
churn                    Int64
revenue_mean           float64
revenue_sum            float64
revenue_max            float64
revenue_min            float64
revenue_std            float64
revenue_last           float64
ownareaall_sqm_mean    float64
ownareaall_sqm_max     float64
ownareaall_sqm_min     float64
ownareaall_sqm_std     float64
ownareaall_sqm_last    float64
format_name_last        object
startdatetime_count      int64
startdatetime_std        int64
count_n                  int64
dtype: object

In [22]:
target_column = ['churn']
cat_columns = ['format_name_last']
id_column = ['customer_id']
feature_columns = list(set(df_train.columns) - set(target_column + id_column))

In [23]:
# train_data = df_train

In [24]:
X_train = train_data[feature_columns]
y_train = train_data[target_column]

X_val = val_data[feature_columns]
y_val = val_data[target_column]

# X_test = test_data[feature_columns]
# y_test = test_data[target_column]

X_train.shape, y_train.shape, X_val.shape, y_val.shape  # , y_test.shape, X_test.shape

((449999, 15), (449999, 1), (50001, 15), (50001, 1))

## Кастомные вспомогательные функции

In [25]:
# Вывод графика ROC-AUC
def plot_roc_auc(y_true, y_pred):
    fpr, tpr, _ = roc_curve(y_true=y_true, y_score=y_pred)
    roc_auc = roc_auc_score(y_true=y_true, y_score=y_pred)

    plt.figure(figsize=(10, 3))
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area = %0.4f)' % roc_auc, alpha=0.5)

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', alpha=0.5)

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('Receiver operating characteristic', fontsize=16)
    plt.legend(loc="lower right", fontsize=12)
    plt.show()
    return roc_auc


In [26]:
# Вывод графика feature importance
def plot_feature_importance(importance, names, model_name="", top_n=-1, skip_columns=[]):
    """
        Функция вывода feature importance
            :importance - массив важности фичей, полученный от модели
            :names - массив названий фичей
            :model_name - название модели
            :top_n - кол-во выводимых фичей
            :skip_columns: какие фичи пропустить, такое может понадобиться чтобы временно убрать 
                            из отображаемых горячие фичи, и изучить менее сильные
            :return - fi_df - feature importance датафрейм
    """
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    fi_df = fi_df[~fi_df['feature_names'].isin(skip_columns)]
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    
    plt.figure(figsize=(10,8))
    sns.barplot(x=fi_df['feature_importance'][:top_n], y=fi_df['feature_names'][:top_n])
    if top_n != -1:
        plt.title(f"{model_name} FEATURE IMPORTANCE (Top: {top_n})")
    else:
        plt.title(f"{model_name} FEATURE IMPORTANCE")
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
    return fi_df


# Baseline

## OverSampling

In [27]:
X_train, y_train = RandomOverSampler(random_state=53).fit_resample(X_train, y_train)
X_train.shape, y_train.shape, X_val.shape, y_val.shape,  # y_test.shape, X_test.shape

# По итогам анализа выявлено, что сэмплирование данных неплохо работает для данного датасета

((646370, 15), (646370, 1), (50001, 15), (50001, 1))

## CatBoost | Отток пользователей (Бинарная классификация)

In [28]:
# Расчет дисбалнса классов
tdist = y_train['churn'].value_counts()
class_weights = {0: tdist[1] / tdist[0], 1: tdist[0] / tdist[1]}

class_weights

{0: 1.0, 1: 1.0}

In [29]:
model = CatBoostClassifier(
    eval_metric="F1",
    iterations=1000,
    early_stopping_rounds=50, 
    class_weights=class_weights, 
    cat_features=cat_columns, 
    random_state=53
)
model.fit(
    X_train, 
    y_train, 
    eval_set=(X_val, y_val), 
    plot=True, verbose=False
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [31]:
# # Для рассчета ROC-AUC на baseline моделе используем тестовые данные
# y_pred_proba = model.predict_proba(X_test)[:, 1]
# y_pred = model.predict(X_test)

In [32]:
# # Строим график ROC-AUC
# roc_auc = plot_roc_auc(y_true=y_test, y_pred=y_pred_proba)
# print("ROC-AUC Score: ", roc_auc)
# print(classification_report(y_test, y_pred))
# print("F1 Score: ", f1_score(y_test, y_pred))

In [33]:
# # Построение важности признаков
# dfi = plot_feature_importance(model.get_feature_importance(), X_test.columns, top_n=30) 

## CatBoost Baseline | Регрессия времени до минимальной будущей транзакции

## Inference

In [34]:
# Считаем признаки через агрегации для тестовой выборки 
df_test = df_test.groupby("customer_id", as_index=False).agg(
    # churn=("buy_post", "last"),
    revenue_mean=("revenue", "mean"),
    revenue_sum=("revenue", "sum"),
    revenue_max=("revenue", "max"),
    revenue_min=("revenue", "min"),
    revenue_std=("revenue", "std"),
    revenue_last=("revenue", "last"),
    ownareaall_sqm_mean=("ownareaall_sqm", "mean"),
    ownareaall_sqm_max=("ownareaall_sqm", "max"),
    ownareaall_sqm_min=("ownareaall_sqm", "min"),
    ownareaall_sqm_std=("ownareaall_sqm", "std"),
    ownareaall_sqm_last=("ownareaall_sqm", "last"),
    format_name_last=("format_name", "last"),
    startdatetime_count=("startdatetime", "count"),
    startdatetime_std=("startdatetime", "std")
)
df_test['startdatetime_std'] = df_test['startdatetime_std'].dt.days
df_test = df_test.merge(test_customer_counts, how='left', on='customer_id')

In [35]:
X_test_final = df_test[feature_columns]

assert X_test_final.shape[1] == X_train.shape[1], "Не совпадают размерности!"

In [36]:
submit['buy_post'] = model.predict(X_test_final)

In [37]:
submit['buy_post'].value_counts()

buy_post
0    59247
1    53087
Name: count, dtype: int64

In [38]:
submit.to_csv("Baseline_GibData.csv", sep=';', index=False)